<a href="https://colab.research.google.com/github/smatiolids/astra-agent-memory/blob/main/YouTube_Knowledge_Base_Text_and_Similarity_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video transcription to fill knowledge base

I talked to a customer who wants to leverage influencers' comments to recommend their products.

The idea then is to process public YouTube videos, collecting influencers' speeches to use their voice to generate responses and indicate the video content that was the response's source.

The Hybrid Search would be necessary to find specific products, categories or product types related to the company business.

I used this exercise as part of a demo I will do for a customer, so I tried to solve both the learning and the demo in one shot.


### References:

https://code.pieces.app/blog/how-to-download-a-youtube-video-in-mp3-format-with-python
https://lablab.ai/t/whisper-tutorial



### Before start

- Set runtime type as GPU
- Upload the .env file
- Upload the secure connect bundle

## Installing Whisper and FFMPEG

In [ ]:
# Install whisper.ai
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-yitn_i9i
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-yitn_i9i
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  

In [ ]:
!sudo apt update && sudo apt install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,231 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,145 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,331 kB]

In [ ]:
import torch
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

# Importing Youtube audio


In [ ]:
!pip install pytube moviepy mock pydub python-dotenv cassio openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 53.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import os
import pytube
from moviepy.editor import *
from pydub import AudioSegment

In [ ]:
def split_mp3(input_file, output_directory, segment_duration_sec, overlap_sec = 1):
    # Load the MP3 file
    audio = AudioSegment.from_mp3(input_file)
    segment_duration_ms = segment_duration_sec * 1000
    overlap_ms = overlap_sec * 1000
    # Calculate the number of segments
    num_segments = len(audio) // ( segment_duration_ms - overlap_ms)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Split the MP3 file into segments
    for i in range(num_segments):
        start_time = i * ( segment_duration_ms - overlap_ms)
        if start_time < 0:
          start_time = 0

        end_time = (i + 1) * segment_duration_ms
        segment = audio[start_time:end_time]

        # Define the output filename
        output_file = os.path.join(output_directory, f"segment_{str(i + 1).zfill(5)}.mp3")

        # Export the segment as an MP3 file
        segment.export(output_file, format="mp3")

        print(f"Segment {i + 1} saved as {output_file}")

    return True



In [ ]:
# Solve pytube issue: https://github.com/pytube/pytube/issues/1498
import re
import mock

from pytube.cipher import get_throttling_function_code

def patched_throttling_plan(js: str):
    """Patch throttling plan, from https://github.com/pytube/pytube/issues/1498"""
    raw_code = get_throttling_function_code(js)

    transform_start = r"try{"
    plan_regex = re.compile(transform_start)
    match = plan_regex.search(raw_code)

    #transform_plan_raw = find_object_from_startpoint(raw_code, match.span()[1] - 1)
    transform_plan_raw = js

    # Steps are either c[x](c[y]) or c[x](c[y],c[z])
    step_start = r"c\[(\d+)\]\(c\[(\d+)\](,c(\[(\d+)\]))?\)"
    step_regex = re.compile(step_start)
    matches = step_regex.findall(transform_plan_raw)
    transform_steps = []
    for match in matches:
        if match[4] != '':
            transform_steps.append((match[0],match[1],match[4]))
        else:
            transform_steps.append((match[0],match[1]))

    return transform_steps

def download_youtube_audio(video_url, output_path):
  try:
    with mock.patch('pytube.cipher.get_throttling_plan', patched_throttling_plan):
      import pytube
      # Create a YouTube object
      yt = pytube.YouTube(video_url)

      # Get the highest resolution stream
      video_stream = yt.streams.filter(only_audio=True).first()

      # Download the audio stream
      video_stream.download(output_path=output_path)

      # Convert the downloaded file to MP3
      mp4_file_path = os.path.join(output_path, video_stream.default_filename)
      mp3_file_path = os.path.splitext(mp4_file_path)[0] + ".mp3"

      video_clip = AudioFileClip(mp4_file_path)
      video_clip.write_audiofile(mp3_file_path)

      # Delete the original MP4 file
      os.remove(mp4_file_path)

      print(f"Audio downloaded and saved as {mp3_file_path}")
      return mp3_file_path
  except Exception as e:
    print(f"An error occurred: {str(e)}")

## Downloading a video from youtube

In [ ]:
audio_file = download_youtube_audio("https://www.youtube.com/watch?v=nk7ccIUTfxU",".")

MoviePy - Writing audio in ./Quinto Andar - Imóvel 698647.mp3


MoviePy - Done.
Audio downloaded and saved as ./Quinto Andar - Imóvel 698647.mp3


## Splitting audio file in chunks of 29 seconds, with an overlap of 1 second

In [ ]:
split_mp3(audio_file,"audio",29)

Segment 1 saved as audio/segment_00001.mp3
Segment 2 saved as audio/segment_00002.mp3
Segment 3 saved as audio/segment_00003.mp3
Segment 4 saved as audio/segment_00004.mp3
Segment 5 saved as audio/segment_00005.mp3
Segment 6 saved as audio/segment_00006.mp3
Segment 7 saved as audio/segment_00007.mp3
Segment 8 saved as audio/segment_00008.mp3
Segment 9 saved as audio/segment_00009.mp3
Segment 10 saved as audio/segment_00010.mp3
Segment 11 saved as audio/segment_00011.mp3
Segment 12 saved as audio/segment_00012.mp3


True

## Transcribing Audio


In [ ]:
# Running whisper from command line
#!whisper "Comentando comentários  Drauzio Comenta 00.mp3" --language Portuguese --model medium

In [ ]:
import whisper
import numpy as np
import pandas as pd

model = whisper.load_model("medium", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 135MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:
#!rm -r ./audio
#!rm ./Comentando*

rm: cannot remove './audio': No such file or directory


In [ ]:
def decode_audio(input_directory):
  import whisper
  import numpy as np

  model = whisper.load_model("medium", device=DEVICE)
  res = []
  for filename in sorted(os.listdir(input_directory)):
    if filename.endswith(".mp3"):
      print(filename)
      audio = whisper.load_audio(f"""{input_directory}/{filename}""" )
      audio = whisper.pad_or_trim(audio)
      mel = whisper.log_mel_spectrogram(audio).to(model.device)
      options = whisper.DecodingOptions(language="pt")
      result = whisper.decode(model, mel, options)
      res.append({
          'segment': filename,
          'text':result.text
      })
  return res


In [ ]:
# print the recognized text
decoded = decode_audio("./audio")

segment_00001.mp3
segment_00002.mp3
segment_00003.mp3
segment_00004.mp3
segment_00005.mp3
segment_00006.mp3
segment_00007.mp3
segment_00008.mp3
segment_00009.mp3
segment_00010.mp3
segment_00011.mp3
segment_00012.mp3


In [ ]:
df = pd.DataFrame(decoded)
df.head()

,segment,text
0,segment_00001.mp3,"Olá, meu nome é Viviane, eu sou fotógrafa parc..."
1,segment_00002.mp3,Aqui tem uma varanda super gostosa Além da rua...
2,segment_00003.mp3,Então aqui é a parte de sala. Eu vou mostrar p...
3,segment_00004.mp3,"pra mostrar os quartos. Aqui tem o lavabo, aqu..."
4,segment_00005.mp3,e aqui toda a extensão de armários. Saindo daq...


# Enter Astra

I would use CassIO, but as it doesn;t have the support to analyzers (yet), I will use regular CQL

In [ ]:
!pip install python-dotenv cassio openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 91.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import os
from dotenv import find_dotenv, load_dotenv
dotenv_file = find_dotenv('.env')
load_dotenv(dotenv_file, override=True)

True

In [ ]:
#import cassio
#cassio.init(token=os.environ["ASTRA_DB_APPLICATION_TOKEN"], database_id=os.environ["ASTRA_DB_ID"])

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(133345985551744) b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.db.astra.datastax.com:29042:e92b9c1c-6c60-4f91-8742-1c43b0145d22> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [ ]:
# Astra Connection (Based on cqlsession, from cassio library)
import os
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

def getCQLSession(mode='astra_db'):
    print('Initializing CQL Session')
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"],
            },
            auth_provider=PlainTextAuthProvider(
                os.environ["ASTRA_DB_CLIENT_ID"],
                os.environ["ASTRA_DB_CLIENT_SECRET"],
            ),
        )
        astraSession = cluster.connect(os.environ["ASTRA_DB_KEYSPACE"])
        astraSession.row_factory = dict_factory
        print('Connected')
        return astraSession
    elif mode == 'local':
        cluster = Cluster()
        localSession = cluster.connect()
        return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    return os.environ["ASTRA_DB_KEYSPACE"]


In [ ]:
# Open the cennection with Astra
session = getCQLSession()

from cassandra.cqlengine.query import BatchStatement
from cassandra.query import BatchType
keyspace = os.environ["ASTRA_DB_KEYSPACE"]
table = f"""yt_navent_audio_texts"""

Initializing CQL Session


ERROR:cassandra.connection:Closing connection <AsyncoreConnection(132965349277072) b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.db.astra.datastax.com:29042:95b6d7f5-9bb3-4560-82a9-f8f5e9363502> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Connected


In [ ]:
cmd_create_table = f"""CREATE TABLE IF NOT EXISTS {table} (
    id text,
    decoded text,
    embedding vector<float, 1536>,
    PRIMARY KEY (id)
)"""

cmd_create_ix_vector = f"""CREATE CUSTOM INDEX IF NOT EXISTS {table}_emb ON {table} (embedding)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{'similarity_function': 'DOT_PRODUCT' }}"""

cmd_create_ix_analyzer = f"""CREATE CUSTOM INDEX IF NOT EXISTS {table}_an ON {table} (decoded)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'WITH OPTIONS = {{ 'index_analyzer': 'brazilian', 'case_sensitive': false}}"""

In [ ]:
cmd_create_ix_analyzer

"CREATE CUSTOM INDEX IF NOT EXISTS yt_navent_audio_texts_an ON yt_navent_audio_texts (decoded)\nUSING 'org.apache.cassandra.index.sai.StorageAttachedIndex'WITH OPTIONS = { 'index_analyzer': 'brazilian', 'case_sensitive': false}"

In [ ]:
session.execute(cmd_create_table)
session.execute(cmd_create_ix_vector)
session.execute(cmd_create_ix_analyzer)

In [ ]:
cmd_insert = f"""
INSERT INTO {table}  (id, decoded, embedding)
VALUES (:id, :decoded, :embedding)
"""
prepared_stmt = session.prepare(cmd_insert)

In [ ]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']

In [ ]:
# Generating embeddings
for index, row in df.iterrows():
  session.execute(prepared_stmt,{'id': f"""{audio_file}_{index}""", 'decoded': row["text"], 'embedding': generate_embedding(row["text"],)})

# Searching mentions

If we want to find mentions of a specific term, like a product, the analyzer works better than the embeddings.

Let's search for mentions of the planet Saturn.

In [ ]:
# Query with parameter
cmd_select_1 = f"""
select id, decoded from {table}
where decoded : :query
limit 5;
"""
prepared_select_1 = session.prepare(cmd_select_1)
rs = session.execute(prepared_select_1,{"query": "jardim"})
list(rs)

[{'id': './Quinto Andar - Imóvel 698647.mp3_9',
  'decoded': 'No total são três quartos, então tem uma suíte e aqui os outros quartos. Dá para um jardim aqui também. Aqui tem um armário ali.'}]

# Running with similarity

Now, let's ask for content related to Saturn based on similarity.

In [ ]:
# Query with parameter
query = "Jardim"

cmd_select_2 = f"""
select id, decoded from {table}
ORDER BY embedding ANN OF :embedding
limit 5;
"""
prepared_select_2 = session.prepare(cmd_select_2)
rs = session.execute(prepared_select_2,{"embedding": generate_embedding(query)})
list(rs)

[{'id': './Quinto Andar - Imóvel 698647.mp3_1',
  'decoded': 'Aqui tem uma varanda super gostosa Além da rua também, isso é bem arborizado, tem uma praça aqui na frente Então aqui é a varanda, aí dá pra ver bem a fachada da casa também'},
 {'id': './Quinto Andar - Imóvel 698647.mp3_9',
  'decoded': 'No total são três quartos, então tem uma suíte e aqui os outros quartos. Dá para um jardim aqui também. Aqui tem um armário ali.'},
 {'id': './Quinto Andar - Imóvel 698647.mp3_0',
  'decoded': 'Olá, meu nome é Viviane, eu sou fotógrafa parceira do Quinto Andar e eu vou mostrar pra vocês essa casa que fica na rua Caminha de Amorim, no bairro de Alto de Pinheiros, cidade de São Paulo. Então aqui a gente tem a porta de entrada social, aí tem sala em dois ambientes, sala de estar e sala de jantar.'},
 {'id': './Quinto Andar - Imóvel 698647.mp3_3',
  'decoded': 'pra mostrar os quartos. Aqui tem o lavabo, aqui a cozinha com bastante armário, aqui tem o lugar pra colocar geladeiras, aqui mesa de a

In this case, we retrieved more information related to Saturn due to the similarity, which found more content related to astronomy.

# Combining both

In this case, I have only three records mentioning Saturn, but we can combine both if I have to choose only one record.

From here, we can add the appropriate docs to prompts/RAG